In [1]:
from utils import *

hete = Heterogeneity()
hete.check_torch_gpu()

-------------------------------------------------
------------------ VERSION INFO -----------------
Conda Environment: torchy | Python version: 3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]
Torch version: 2.0.1
Torch build with CUDA? True
# Device(s) available: 1, Name(s): Quadro P520



***
# END

In [290]:
class Generator(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(Generator, self).__init__()
        # Define the architecture of the generator
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, output_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        # Forward pass of the generator
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [291]:
class Discriminator(nn.Module):
    def __init__(self, input_channels):
        super(Discriminator, self).__init__()
        # Define the architecture of the discriminator
        self.model = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Forward pass of the discriminator
        output = self.model(x)
        return output

In [289]:
x0 = np.moveaxis(np.moveaxis(X_data, -1, 1), -1, 1).reshape(20*61,4,64,64)
y0 = np.moveaxis(np.moveaxis(y_data, -1, 1), -1, 1).reshape(20*61,2,64,64)

X = torch.from_numpy(x0).float()
y = torch.from_numpy(y0).float()

In [306]:
G_XY = Generator(input_channels=4, output_channels=2)
G_YX = Generator(input_channels=2, output_channels=4)

D_X = Discriminator(input_channels=4)
D_Y = Discriminator(input_channels=2)

In [307]:
criterion_GAN   = nn.MSELoss()  
criterion_cycle = nn.L1Loss() 

optimizer_G   = Adam(list(G_XY.parameters()) + list(G_YX.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_X = Adam(D_X.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_Y = Adam(D_Y.parameters(), lr=0.0002, betas=(0.5, 0.999))

num_epochs = 10
batch_size = 2

In [308]:
for epoch in range(num_epochs):
    for i in range(len(X)):
        real_X = Variable(X[i]).unsqueeze(0)
        real_Y = Variable(y[i]).unsqueeze(0)

        # --------------------
        # Training generators
        # --------------------

        optimizer_G.zero_grad()

        # Identity loss
        loss_identity_X = criterion_cycle(G_YX(real_X), real_X)
        loss_identity_Y = criterion_cycle(G_XY(real_Y), real_Y)

        # GAN loss
        fake_Y = G_XY(real_X)
        pred_fake_Y = D_Y(torch.cat((real_X, fake_Y), 1))
        loss_GAN_XY = criterion_GAN(pred_fake_Y, torch.ones(pred_fake_Y.size()))

        fake_X = G_YX(real_Y)
        pred_fake_X = D_X(torch.cat((real_Y, fake_X), 1))
        loss_GAN_YX = criterion_GAN(pred_fake_X, torch.ones(pred_fake_X.size()))

        # Cycle consistency loss
        recovered_X = G_YX(fake_Y)
        loss_cycle_X = criterion_cycle(recovered_X, real_X)

        recovered_Y = G_XY(fake_X)
        loss_cycle_Y = criterion_cycle(recovered_Y, real_Y)

        # Total generator loss
        loss_G = loss_identity_X + loss_identity_Y + loss_GAN_XY + loss_GAN_YX + lambda_cycle * (loss_cycle_X + loss_cycle_Y)

        # Backward and optimize
        loss_G.backward()
        optimizer_G.step()

        # ------------------------
        # Training discriminators
        # ------------------------

        optimizer_D_X.zero_grad()

        # Real loss
        pred_real_X = D_X(torch.cat((real_Y, real_X), 1))
        loss_D_X_real = criterion_GAN(pred_real_X, torch.ones(pred_real_X.size()))

        # Fake loss
        fake_X = G_YX(real_Y)
        pred_fake_X = D_X(torch.cat((real_Y, fake_X.detach()), 1))
        loss_D_X_fake = criterion_GAN(pred_fake_X, torch.zeros(pred_fake_X.size()))

        # Total discriminator X loss
        loss_D_X = (loss_D_X_real + loss_D_X_fake) * 0.5

        # Backward and optimize
        loss_D_X.backward()
        optimizer_D_X.step()

        optimizer_D_Y.zero_grad()

        # Real loss
        pred_real_Y = D_Y(torch.cat((real_X, real_Y), 1))
        loss_D_Y_real = criterion_GAN(pred_real_Y, torch.ones(pred_real_Y.size()))

        # Fake loss
        fake_Y = G_XY(real_X)
        pred_fake_Y = D_Y(torch.cat((real_X, fake_Y.detach()), 1))
        loss_D_Y_fake = criterion_GAN(pred_fake_Y, torch.zeros(pred_fake_Y.size()))

        # Total discriminator Y loss
        loss_D_Y = (loss_D_Y_real + loss_D_Y_fake) * 0.5

        # Backward and optimize
        loss_D_Y.backward()
        optimizer_D_Y.step()

        # Print the losses
        if (i + 1) % print_freq == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(X)}], Loss_D_X: {loss_D_X.item():.4f}, Loss_D_Y: {loss_D_Y.item():.4f}, Loss_G: {loss_G.item():.4f}")

    # Save the generated images
    if (epoch + 1) % save_freq == 0:
        with torch.no_grad():
            fake_Y = G_XY(real_X)
            fake_X = G_YX(real_Y)
            save_image(fake_Y, f"generated_images/fake_Y_{epoch+1}.png", normalize=True)
            save_image(fake_X, f"generated_images/fake_X_{epoch+1}.png", normalize=True)

    # Print the epoch and save the models
    if (epoch + 1) % save_freq == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] completed. Saving models...")
        torch.save(G_XY.state_dict(), f"models/G_XY_{epoch+1}.pt")
        torch.save(G_YX.state_dict(), f"models/G_YX_{epoch+1}.pt")
        torch.save(D_X.state_dict(), f"models/D_X_{epoch+1}.pt")
        torch.save(D_Y.state_dict(), f"models/D_Y_{epoch+1}.pt")

RuntimeError: Given groups=1, weight of size [64, 2, 4, 4], expected input[1, 4, 64, 64] to have 2 channels, but got 4 channels instead

In [162]:
enc = Encoder()
dec = Decoder()

x = torch.Tensor(np.moveaxis(np.moveaxis(X_data, -2, 1).reshape(20*61, 64, 64, 4), -1, 1))
print('x:', x.shape)

z = enc(x)
print('z:', z.shape)

y = dec(z)
print('y:', y.shape)

x: torch.Size([1220, 4, 64, 64])
z: torch.Size([1220, 500])


TypeError: expected Tensor as element 0 in argument 0, but got Sequential

In [72]:
        self.up_transpose_1   = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.up_convolution_1 = double_convolution(256, 128)
        self.up_transpose_2   = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.up_convolution_2 = double_convolution(128, 64)
        self.up_transpose_3   = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.up_convolution_3 = double_convolution(64, 32)
        self.up_transpose_4   = nn.ConvTranspose2d(32, 16, kernel_size=2,  stride=2)
        self.up_convolution_4 = double_convolution(32, 16)
        self.out = nn.Conv2d(16, 2, kernel_size=1) 
        
    def forward(self, x):
        down_1 = self.down_convolution_1(x)
        down_2 = self.max_pool2d(down_1)
        down_3 = self.down_convolution_2(down_2)
        down_4 = self.max_pool2d(down_3)
        down_5 = self.down_convolution_3(down_4)
        down_6 = self.max_pool2d(down_5)
        down_7 = self.down_convolution_4(down_6)
        down_8 = self.max_pool2d(down_7)
        down_9 = self.down_convolution_5(down_8)   
             
        up_1 = self.up_transpose_1(down_9)
        x    = self.up_convolution_1(torch.cat([down_7, up_1], 1))
        up_2 = self.up_transpose_2(x)
        x    = self.up_convolution_2(torch.cat([down_5, up_2], 1))
        up_3 = self.up_transpose_3(x)
        x    = self.up_convolution_3(torch.cat([down_3, up_3], 1))
        up_4 = self.up_transpose_4(x)
        x    = self.up_convolution_4(torch.cat([down_1, up_4], 1))
        out   = self.out(x)
        return out

In [68]:
X_data = np.load('X_data.npy')
y_data = np.load('y_data.npy')
print(X_data.shape, y_data.shape)

(20, 64, 64, 61, 4) (20, 64, 64, 61, 2)


In [ ]:
device = 'cuda'

rom = h2_hete_rom().to(device)
optimizer = NAdam(rom.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

X_train = torch.Tensor(X_data).to(device)
y_train = torch.Tensor(y_data).to(device)

loss, val_loss = [], []
metrics = {'loss':[], 'val_loss':[]}
epochs = 10
batch_size = 1

for epoch in range(epochs):
    rom.train()
    epoch_loss = 0.0
    for i in range(0, len(X_train), batch_size):
        inp  = X_train[i:i+batch_size]
        true = y_train[i:i+batch_size]
        optimizer.zero_grad()
        pred = rom(inp)
        loss = loss_fn(pred,true)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()*inp.size(0)
    metrics['loss'].append(epoch_loss/len(X_train))